# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.utils import shuffle

In [3]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords

In [4]:
import re
from pymystem3 import Mystem

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#### Загружаем датасет

In [7]:
df_comments = pd.read_csv('toxic_comments.csv')

Очищаем данные от лишних символов

In [8]:
def clear_text(row):
    text = row['text']
    clear_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text_list = clear_text.split()
    return " ".join(text_list)

In [9]:
df_comments['clear_text'] = df_comments.apply(clear_text, axis=1)

Загружаем стоп-слова

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Димачка-
[nltk_data]     хороший\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Сформируем фичи и целевой признак, а также разобьем датасет на обучающую и тестовые выборки.

In [11]:
features = df_comments['clear_text']
target = df_comments['toxic']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)

Создаем счётчик, указав в нём стоп-слова

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, max_features=10000)

Рассчитаем TF-IDF для корпуса текстов

In [14]:
X_train_tf_idf = count_tf_idf.fit_transform(X_train)

In [15]:
X_test_tf_idf = count_tf_idf.transform(X_test)

# 2. Обучение

### Логистическая регрессия

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
model_lr = LogisticRegression(solver='lbfgs')

Обучаем логистическую регрессию на рассчитанных TF-IDF для обучающей выборки

In [18]:
%%time
model_lr.fit(X_train_tf_idf, y_train)

Wall time: 2.24 s


LogisticRegression()

In [19]:
pred_lr_train = model_lr.predict(X_train_tf_idf)

In [20]:
f1_score(y_train, pred_lr_train)

0.7686166995927911

#### Метрика качества F1 для логистической регрессии на обучающей выборке равна 0.768.

### Модель LightGBM

In [21]:
import lightgbm

In [22]:
model_lgbm = lightgbm.LGBMClassifier()

Обучаем LightGBM на рассчитанных TF-IDF для обучающей выборки

In [23]:
%%time
model_lgbm.fit(X_train_tf_idf, y_train)

Wall time: 37.3 s


LGBMClassifier()

In [24]:
pred_lgbm_train = model_lgbm.predict(X_train_tf_idf)

In [25]:
f1_score(y_train, pred_lgbm_train)

0.7751486373249271

#### Метрика качества F1 для LightGBM на обучающей выборке равна 0.775.

### Модель CatBoostClassifier

In [26]:
from catboost import CatBoostClassifier

In [27]:
cbc = CatBoostClassifier()

Обучаем CatBoostClassifier на рассчитанных TF-IDF для обучающей выборки

In [28]:
%%time
cbc.fit(X_train_tf_idf, y_train)

Learning rate set to 0.085912
0:	learn: 0.6053392	total: 1.33s	remaining: 22m 13s
1:	learn: 0.5347226	total: 2.79s	remaining: 23m 10s
2:	learn: 0.4779582	total: 3.94s	remaining: 21m 47s
3:	learn: 0.4318068	total: 5.19s	remaining: 21m 32s
4:	learn: 0.3962390	total: 6.4s	remaining: 21m 13s
5:	learn: 0.3667762	total: 7.64s	remaining: 21m 5s
6:	learn: 0.3429962	total: 8.77s	remaining: 20m 44s
7:	learn: 0.3249089	total: 9.99s	remaining: 20m 38s
8:	learn: 0.3087351	total: 11.1s	remaining: 20m 23s
9:	learn: 0.2954161	total: 12.4s	remaining: 20m 25s
10:	learn: 0.2835259	total: 13.6s	remaining: 20m 22s
11:	learn: 0.2741709	total: 14.9s	remaining: 20m 26s
12:	learn: 0.2668610	total: 16s	remaining: 20m 17s
13:	learn: 0.2603705	total: 17.3s	remaining: 20m 20s
14:	learn: 0.2548994	total: 18.5s	remaining: 20m 11s
15:	learn: 0.2500178	total: 19.7s	remaining: 20m 11s
16:	learn: 0.2462112	total: 20.8s	remaining: 20m 3s
17:	learn: 0.2424774	total: 22.1s	remaining: 20m 3s
18:	learn: 0.2394477	total: 23.2

In [29]:
pred_cbc_train = cbc.predict(X_train_tf_idf)

In [30]:
f1_score(y_train, pred_cbc_train)

0.8014934171742975

#### Метрика качества F1 для CatBoostClassifier на обучающей выборке равна 0.801.

Так как полученная разница метрики F1 для разных моделей не велика, проведем тестирование для всех трех моделей.

### Тестирование

#### Логистическая регрессия

In [31]:
pred_lr = model_lr.predict(X_test_tf_idf)

In [32]:
f1_score(y_test, pred_lr)

0.7606581899775617

#### Модель LightGBM

In [33]:
pred_lgbm = model_lgbm.predict(X_test_tf_idf)

In [34]:
f1_score(y_test, pred_lgbm)

0.7510265024262784

#### Модель CatBoostClassifier

In [35]:
pred_cbc = cbc.predict(X_test_tf_idf)

In [36]:
f1_score(y_test, pred_cbc)

0.7677585572322414

# 3. Выводы

Для возможности решения задачи классификации комментариев как токсичных, тексты были очищены от лишних символов. Для каждого текста рассчитано TF-IDF.  
Проведено обучение и тестирование трех моделей: логистической регрессии, LightGBM, CatBoostClassifier.  
#### Лучший результат на тестировании показала модель CatBoostClassifier, метрика F1 равна 0.767. Второй по качеству стала модель логистической регресии, F1 равно 0.76.  
#### При этом время обучения модели логистической регресии меньше.  
#### Модели логистической регрессии и CatBoostClassifier, могут быть использованы для определения токсичности текстов. При этом, если важно время обучения, то предпочтительней использовать логистическую регрессию.